In [1]:
from transformers import OpenAIGPTTokenizer, OpenAIGPTModel, OpenAIGPTLMHeadModel, AdamW, DataCollatorForLanguageModeling
import re
import json
import torch
from transformers import LineByLineTextDataset
from sklearn.model_selection import train_test_split

torch.cuda.empty_cache()

tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
dataset = LineByLineTextDataset(tokenizer, 'imdb_cache.txt', 40)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
/home/ubuntu/anaconda3/envs/pytorch_p37/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [8]:
from transformers import Trainer, TrainingArguments,AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./gpt2-out", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    eval_dataset=dataset,
)

loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /home/ubuntu/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation

In [9]:
trainer.train()

***** Running training *****
  Num examples = 10001
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 939


Step,Training Loss
500,5.531500


Saving model checkpoint to ./gpt2-out/checkpoint-800
Configuration saved in ./gpt2-out/checkpoint-800/config.json
Model weights saved in ./gpt2-out/checkpoint-800/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=939, training_loss=4.94155107037074, metrics={'train_runtime': 721.5534, 'train_samples_per_second': 41.581, 'train_steps_per_second': 1.301, 'total_flos': 612464440320000.0, 'train_loss': 4.94155107037074, 'epoch': 3.0})

In [10]:
trainer.save_model()

Saving model checkpoint to ./gpt2-out
Configuration saved in ./gpt2-out/config.json
Model weights saved in ./gpt2-out/pytorch_model.bin


In [11]:
from transformers import pipeline

finetune = pipeline('text-generation',model='./gpt2-out', tokenizer='openai-gpt')

loading configuration file ./gpt2-out/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.12.3",
  "use_cache": true,
  "vocab_size": 50257
}

loading configurat

In [12]:
finetune('THIS IS A MOVIE REVIEW. REVIEW:')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'THIS IS A MOVIE REVIEW. REVIEW: pos start review : this is one of those well the film on the funniest of all, in this movie, in a film of a movie, " on the movie " i loved " the characters, this movie'}]

In [13]:
finetune('THIS IS A MOVIE REVIEW.')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "THIS IS A MOVIE REVIEW. review : neg start review : i don't know a long of time and the film is an the very people. i have ever seen to see i can't really, and i have no get the big"}]

In [15]:
finetune('hello, how ')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "hello, how  not but a true one of my went - about a film of the 19! < br / > < br / > i'm a lot on there are bad to watch with - - and it ( ) to be the first."}]

In [16]:
import pickle

with open("../rollouts.pkl", 'rb') as f:
    rollouts = pickle.load(f)

In [19]:
rollouts[0].hindsight_trajectory()

[["\n\n\nYou don't like doors? Why not try going north, that entranceway is unblocked.\n",
  "\n-= Roomy Garage =-\nYou find yourself in a garage. A roomy one.\n\n\n\nYou don't like doors? Why not try going north, that entranceway is unguarded. You need an unblocked exit? You should try going south.\n\n",
  'north'],
 ['\n-= Garage =-\nYou arrive in a garage. An ordinary kind of place. You try to gain information on your surroundings by using a technique you call "looking."\n\n\n\nThere is an exit to the east. Don\'t worry, it is unguarded. There is an unblocked exit to the south.\n\n',
  "\n-= Roomy Garage =-\nYou find yourself in a garage. A roomy one.\n\n\n\nYou don't like doors? Why not try going north, that entranceway is unguarded. You need an unblocked exit? You should try going south.\n\n",
  'east'],
 ["\n-= Cellar =-\nThis is going to sound unbelievable, but you've just entered a cellar.\n\n\n\nYou don't like doors? Why not try going south, that entranceway is unblocked. You 